In [19]:
from importlib import reload
import dataprep_functions as f

In [26]:
f = reload(f)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
census_df = f.merge_census(f.census_1, f.census_2)

In [7]:
# f.community_detection_iters()

In [8]:
# f.calc_consen()

In [9]:
tract_df = f.create_tract_df_with_degree()

In [10]:
tract_df, community_df = f.create_community_df(tract_df)

In [11]:
tract_df = f.drop_duplicate_tracts(tract_df)

In [27]:
community_df, com_mod_df = f.calc_community_modularity(community_df, tract_df)

/home/barcsab/urban_communities/scripts/dataprep_functions.py:256: RuntimeWarning: invalid value encountered in true_divide
  det = det / normadet


ValueError: operands could not be broadcast together with shapes (2176,2176) (2171,2171) 

In [ ]:
network_df = f.calc_modularity(com_mod_df)

In [ ]:
community_df = f.calc_modularity_contribution(community_df, network_df)

In [ ]:
community_df, network_df = f.calc_tract_number(tract_df, community_df, network_df, f.cbsacode)

In [ ]:
network_df = f.calc_network_density(network_df)

In [ ]:
network_df = f.calc_community_county_similarity(tract_df)

In [ ]:
tract_df, census_df = f.clean_census_to_tract_df(tract_df, census_df)

In [ ]:
community_df, network_df = f.calc_sum_and_std(tract_df, community_df, network_df)

In [ ]:
tract_df, community_df = f.calc_income_percentile(tract_df, community_df)

In [ ]:
tract_df = f.create_poor_dummy(tract_df)

In [ ]:
tract_df = f.diff_from_network_avg(tract_df)

## Save tables

In [ ]:
# tisztitás

In [2]:
%matplotlib inline



# import matplotlib.pyplot as plt


#import seaborn as sns
#import networkx as nx

# import scipy
import csv

import matplotlib.cm as cm


import community as community_louvain
from copy import deepcopy

from itertools import product
import networkx.algorithms.community as nx_comm
from scipy.spatial.distance import pdist, squareform

#from shapely.geometry import Point, LineString
from geopandas import GeoDataFrame

#import matplotlib.lines as lines

In [ ]:
# kiszámolom a S_cons-t és ki fájlba

In [ ]:
beolvasom

In [ ]:
hozzáad

ezt is kiment

In [ ]:
cbsacode = pd.read_csv("../data/cbsacode_shortname_tracts.csv",sep=";", index_col=0)

cbsacode['clean_name'] = cbsacode["short_name"].map(lambda s: s.split("/")[0].replace(' ','_').replace('.','').lower())

In [ ]:
# KIMENŐ EZ LESZ MAJD, CSAK HIBA NÉLÜLI


# Properties of tracts

tract_df = pd.read_csv('../data/network_properties.csv')


In [ ]:
# calculate proportions - mean tract population is not the same among communities

# for 2012
tract_df['educ_bach_p_1'] = tract_df['education_bachelor_1']/tract_df['population_1']
tract_df['educ_total_p_1'] = tract_df['education_total_1']/tract_df['population_1']
tract_df['white_p_1'] = tract_df['white_1']/tract_df['population_1']
tract_df['black_p_1'] = tract_df['black_1']/tract_df['population_1']
tract_df['native_p_1'] = tract_df['native_1']/tract_df['population_1']
tract_df['asian_p_1'] = tract_df['asian_1']/tract_df['population_1']

# for 2017
tract_df['educ_bach_p_2'] = tract_df['education_bachelor_2']/tract_df['population_2']
tract_df['educ_total_p_2'] = tract_df['education_total_2']/tract_df['population_2']
tract_df['white_p_2'] = tract_df['white_2']/tract_df['population_2']
tract_df['black_p_2'] = tract_df['black_2']/tract_df['population_2']
tract_df['native_p_2'] = tract_df['native_2']/tract_df['population_2']
tract_df['asian_p_2'] = tract_df['asian_2']/tract_df['population_2']

In [ ]:
tract_once = tract_df[tract_df['algorithm_type'] == 'mg']
network_df = deepcopy(tract_once.groupby(['city','g_type'])['educ_bach_p_1','white_p_1','black_p_1','native_p_1','asian_p_1', 'income_1', 
                                                          'educ_bach_p_2','white_p_2','black_p_2','native_p_2','asian_p_2', 'income_2'].mean().reset_index())

# every tract is counted twice - once for each algorithm_type -- solved
network_df = network_df.rename(columns = {'educ_bach_p_1' : 'educ_bach_p_avg_1','white_p_1' : 'white_p_avg_1', 'black_p_1' : 'black_p_avg_1', 'native_p_1' : 'native_p_avg_1', 'asian_p_1' : 'asian_p_avg_1', 'income_1' : 'income_avg_1',
                                          'educ_bach_p_2' : 'educ_bach_p_avg_2','white_p_2' : 'white_p_avg_2', 'black_p_2' : 'black_p_avg_2', 'native_p_2' : 'native_p_avg_2', 'asian_p_2' : 'asian_p_avg_2', 'income_2' : 'income_avg_2'})